In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# EDA on the train data
-------

In [ ]:
train_ = pd.read_csv('../input/spaceship-titanic/train.csv')
test_ = pd.read_csv('../input/spaceship-titanic/test.csv')
train = train_.copy()
test = test_.copy()
train.info()

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

# Observing the test data
-------

In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
test.isnull().sum()

# Preprocessing
-----

In [ ]:
#  Creating three new columns based on Cabin Feature such as Deck, Num and Side. 
# This feautes might help me to fill missing values 
train['CabinD'] = train.Cabin.str.split('/').str[0]
train['CabinN'] = train.Cabin.str.split('/').str[1]
train['CabinS'] = train.Cabin.str.split('/').str[2]

test['CabinD'] = test.Cabin.str.split('/').str[0]
test['CabinN'] = test.Cabin.str.split('/').str[1]
test['CabinS'] = test.Cabin.str.split('/').str[2]

# Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with.
# People in a group are often family members. 
# Takes group out might help to prediction. 

train['PassengerId'] = train.PassengerId.str.split('_').str[0]
test['PassengerId'] = test.PassengerId.str.split('_').str[0]

# Since we've create three new columns (Deck, Num and Side), so the original column 'Cabin' is useless.
train = train.drop('Cabin',axis=1)
test = test.drop('Cabin',axis=1)

In [ ]:
# use LabelEncoder to transform non-numerical labels to numerical labels 

y_train = train.pop('Transported')
y_train = y_train.map(lambda x: 1 if x==True else 0)

le = LabelEncoder()

cols = train.select_dtypes('object').columns
comb = train[cols].append(test[cols])

for col in comb.columns:
    comb[col] = le.fit(comb[col])
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [ ]:
# fill nan-value 
train = train.fillna(train.mean())
test = test.fillna(test.mean())

x_train = train
x_test = test

# Model & Prediction
-------

use RandomForestClassifier Model to predict the score

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# xtrain,xtest,ytrain,ytest=train_test_split(X_train,y_train,test_size=0.3)
RFC = RandomForestClassifier (n_estimators=100,random_state=0)
score = cross_val_score(RFC,x_train,y_train,cv=10).mean()
score

In [ ]:
RFC = RFC.fit(x_train,y_train)
ypredict = RFC.predict(x_test)
ypredict

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_["PassengerId"],
        "Transported": ypredict
    })


submission['Transported'] = submission['Transported'].astype(bool)

submission.to_csv('submission.csv', index=False)
submission